SCRIPT FOCUSED ON DOWNLOAD THE ANNUAL DATA FROM THE CFTC OFFICIAL WEBSITE AND INSERT INTO A SQL DATABASE

Import libraries

In [1]:
import requests
import pyodbc
from zipfile import ZipFile
import pandas as pd
import os

Define base URL

In [2]:
base_url = "https://www.cftc.gov"

Define list of years to download

In [3]:
years_to_download = [2022, 2023]

Server database credentials

In [4]:
server = 'DESKTOP-HK9LHN8'
database = 'projetos'
username = 'sa'
password = '1620meguie'

Create connection, unzip files and insert into SQL

In [ ]:
connection_string = f"DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

for year in years_to_download:
    try:
        # Construct URL for the ZIP file (I had to search inside the website for the file repository)
        zip_url = f"{base_url}/files/dea/history/dea_fut_xls_{year}.zip"
        
        # Download the ZIP file
        response = requests.get(zip_url)
        with open(f"dea_fut_xls_{year}.zip", "wb") as zip_file:
            zip_file.write(response.content)
        
        # Extract the XLS file from the ZIP
        with ZipFile(f"dea_fut_xls_{year}.zip", "r") as zip_ref:
            zip_ref.extract("annual.xls", path=".")
        
        # Read the XLS file into a DataFrame
        xls_file = pd.read_excel("annual.xls")
        
        # Name the table based on it year
        table_name = f"COTAnnualData{year}"
        
        # Extract column names from the XLS file
        column_names = [str(col) for col in xls_file.columns]
        
        # Generate the SQL statement to create or replace the table
        create_table_sql = f"""
        IF OBJECT_ID('{table_name}', 'U') IS NOT NULL
            DROP TABLE {table_name}
        CREATE TABLE {table_name} ({', '.join([f'[{col}] NVARCHAR(MAX)' for col in column_names])})
        """
        
        # Execute the CREATE OR REPLACE TABLE statement
        cursor.execute(create_table_sql)
        
        # Insert data into the SQL table with error handling
        for index, row in xls_file.iterrows():
            values = [str(value) for value in row]
            placeholders = ', '.join(['?'] * len(column_names))
            insert_sql = f"INSERT INTO {table_name} ({', '.join(column_names)}) VALUES ({placeholders})"
            try:
                cursor.execute(insert_sql, values)
            except Exception as e:
                print(f"Error inserting data for {year}: {e}")
        
        # Commit the changes to the database
        conn.commit()
        
        # Clean up: remove downloaded files for the current year
        os.remove(f"dea_fut_xls_{year}.zip")
        os.remove("annual.xls")
        
        print(f"Data for {year} successfully downloaded and inserted into the database.")
    except Exception as e:
        print(f"Error for {year}: {e}")

# Close the database connection
conn.close()

Data for 2022 successfully downloaded and inserted into the database.
